# Librerias

In [2]:
!pip install selenium
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Funciones

In [3]:
def try_click_info(info):  # Intentar 
    try:
        print('try 1')
        info[1].click()
    except:
        try:
            print('try 2')
            info[0].click()
        except:
            try:
                print('try 3')
                info[3].click()
            except:
                
                print('try 3 (uniques)')                
                try:
                    info1 = driver.find_element( "xpath",'//a[@class="'+"text-success"+'"]')
                    info1.click()
                except:
                    
                    try:
                        info2 = driver.find_elements( "xpath",'//a[@class="'+"text-muted"+'"]')
                        info2.click()                    
                    except:
                        info = driver.find_elements( "xpath",'//*[@id="rmTable2"]/tbody/tr[2]/td/ul/li[6]/span[2]/a') #<a
                        for i in range(len(info)):
                            info[i].click()

# Nits

In [3]:
#NITs a consultar
NIT = ['900480569', '860013570','800007813','800255754','900092385','860063875',
       '890903938','800141629','860043896','900184722','900881599','900690807'] #NIT

In [4]:
archivo = "nits_8032023.xlsx"#"nits_7032023.xlsx"
sheet_name = 'Hoja2'
df = pd.read_excel(io = archivo, sheet_name = sheet_name )
df = df.drop_duplicates(subset='NIT').reset_index(drop = True)

NIT = list(df.NIT)
print(len(NIT))

174


# Ejecucion

In [5]:
actividades = []
nits = []
no_nits = []
fechas = []
l=1

for n in range(len(NIT)):
    inicio = time.time()
    #Activar WebDriver
    
    print('Activando Web Driver...')
    url='https://www.rues.org.co/'
    options = webdriver.ChromeOptions()
    #options.add_argument('--incognito')
    driver = webdriver.Chrome(executable_path='../chromedriver', options=options)
    driver.get(url)
    
    time.sleep(3) #DELAY
    
    print('Cerrando Ventana Emergente...')
    #Cerrar Ventana Emergente Inicialabs
    button = driver.find_element("xpath", '/html/body/div[1]/section/div/div/div/div[3]/button')
    button.click()

    time.sleep(1)
    
    print('Ingresando Nit ', NIT[n])
    #Caja Ingreso Nit
    input_nit = driver.find_element("xpath", '/html/body/div[1]/main[1]/div/div[2]/div/div/div[2]/form/div/input') # Caja NIT
    input_nit.click() # CLICK

    time.sleep(1)

    input_nit.send_keys(NIT[n]) # INGRESO EL NIT
    driver.find_element("xpath",'/html/body/div[1]/main[1]/div/div[2]/div/div/div[2]/form/button').click() # Click en buscar
    
    time.sleep(1)

    try:
        
        print('Seleccionando Razon Social...')
        # Seleccionar Primer Nombre o Razon Social Resultante
        driver.find_element("xpath",'/html/body/div[1]/main[1]/div/div[6]/div/div/div/div/table/tbody/tr/td[1]').click() 

        time.sleep(1)
        
        #Click en Info
        # Encontrar y clickear info
        try:
            print('Probando Paths 1...')
            info = driver.find_elements( "xpath",'//a[@class="'+"text-success"+'"]') #<a class="text-success" href="javascript:ConsultarDetalleRM('40002161982')"><i class="fa fa-info text-success"></i>nfo</a>
            time.sleep(1)
            try_click_info(info)
        except:
            try:
                print('Probando Paths 2...')
                info2 = driver.find_elements( "xpath",'//a[@class="'+"text-muted"+'"]')
                time.sleep(1)                         
                try_click_info(info2)
            except: 
                print('Path info de ', NIT[n], 'no encontrado')
       
        #Extraer Primera Actividad Economica
        try:
            print('Extrayendo Actividad Economica...')
            actividad = driver.find_element('xpath','/html/body/div[1]/main/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/ul/li[1]')
        
        except: 
            time.sleep(1)
            print('Extrayendo Actividad Economica...')
            actividad = driver.find_element('xpath','/html/body/div[1]/main/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/ul/li[1]')
        
        try:
            print('Extrayendo fecha de Matricula...')
            fecha_matricula = driver.find_element('xpath','/html/body/div[1]/main/div/div[2]/div[2]/div/div[1]/div/div[2]/div/table/tbody/tr[4]/td[2]')
            fecha = fecha_matricula.text
        except:
            print('Fecha de ',NIT[n], 'No encontrada.')
        time.sleep(1)
        
        txt = actividad.text
        actividades.append(txt)
        nits.append(NIT[n])
        fechas.append(fecha)
        
        print('Guardando actividad, nit y fecha')
        print('Actividad: ', actividad.text) 
    except:
        no_nits.append(NIT[n])  
        print(NIT[n],'no encontrado')

    driver.close()
    time.sleep(1)
    
    fin = time.time() #Tiempo 
    print('Duracion de la consulta ',fin-inicio, ' s')
    print('\n')

print('\n')
print('Cantidad de NITS encontrados:               ', len(nits),       '\n',
      'Cantidad de fechas de matriculas guardadas: ', len(fechas),     '\n',
      'Cantidad de Actividades almacenadas:        ', len(actividades),'\n',
      'Cantidad de Registros No encontrados:       ', len(no_nits),    '\n')

print('Los nits ', no_nits, ' no se encuentran en el RUES')
print('\n','Los nits ', nits, ' si se encuentran en el RUES')
print('\n', 'Actividades Extraidas: ', actividades)


# Data 2
df_actividades = pd.DataFrame({'Nit':nits,
                               'Actividades':actividades,
                               'Fecha Matricula':fechas})



df_actividades.to_excel('Rues.xlsx')
df_actividades



Activando Web Driver...


C:\Users\lcast\AppData\Local\Temp\ipykernel_11288\471087958.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../chromedriver', options=options)


Cerrando Ventana Emergente...
Ingresando Nit  79577616
Seleccionando Razon Social...
79577616 no encontrado
Duracion de la consulta  15.194236755371094  s


Activando Web Driver...
Cerrando Ventana Emergente...
Ingresando Nit  8300312240
Seleccionando Razon Social...
8300312240 no encontrado
Duracion de la consulta  15.796975135803223  s


Activando Web Driver...
Cerrando Ventana Emergente...
Ingresando Nit  9011174933
Seleccionando Razon Social...
9011174933 no encontrado
Duracion de la consulta  14.132224082946777  s


Activando Web Driver...
Cerrando Ventana Emergente...
Ingresando Nit  10135782180
Seleccionando Razon Social...
10135782180 no encontrado
Duracion de la consulta  16.602899312973022  s


Activando Web Driver...
Cerrando Ventana Emergente...
Ingresando Nit  17124624
Seleccionando Razon Social...
17124624 no encontrado
Duracion de la consulta  13.833197116851807  s


Activando Web Driver...
Cerrando Ventana Emergente...
Ingresando Nit  9011357385
Seleccionando Razon Soci

ElementClickInterceptedException: Message: element click intercepted: Element <input type="number" name="NIT" id="txtNIT" class="form-control" placeholder="Número de Identificación" value="" maxlength="15" size="20" min="999" max="999999999999999" style="resize:horizontal; width:200px" required=""> is not clickable at point (722, 18). Other element would receive the click: <div class="mn-navigation">...</div>
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x009737D3]
	(No symbol) [0x00908B81]
	(No symbol) [0x0080B36D]
	(No symbol) [0x00844E3B]
	(No symbol) [0x008426DB]
	(No symbol) [0x0083FD0B]
	(No symbol) [0x0083E4D8]
	(No symbol) [0x00833253]
	(No symbol) [0x0085B41C]
	(No symbol) [0x00832B96]
	(No symbol) [0x0085B774]
	(No symbol) [0x00871215]
	(No symbol) [0x0085B216]
	(No symbol) [0x00830D97]
	(No symbol) [0x0083253D]
	GetHandleVerifier [0x00BEABF2+2510930]
	GetHandleVerifier [0x00C18EC1+2700065]
	GetHandleVerifier [0x00C1C86C+2714828]
	GetHandleVeri

In [16]:
# Data 2
df_actividades = pd.DataFrame({'Nit':nits,
                               'Actividades':actividades,
                               'Fecha Matricula':fechas})



df_actividades.to_excel('Actividades_consultadas_0_136.xlsx')
df_actividades


,Nit,Actividades,Fecha Matricula
0,2970862,"5221 Actividades de estaciones, vías y servic...",20080114
1,3047552,"0090 Rentistas de Capital, sólo Personas Natur...",20070719
2,10220988,6810 Actividades inmobiliarias realizadas con ...,20030411
3,11406189,4711 Comercio al por menor en establecimientos...,19971107
4,11450198,4771 Comercio al por menor de prendas de vesti...,20080813
5,19312869,"0090 Rentistas de Capital, sólo Personas Natur...",19800416
6,19350776,,20000410
7,20389932,,00000000
8,30344726,,00000000
9,39728252,4664 Comercio al por mayor de productos químic...,20030226


In [15]:
print('Cantidad de NITS encontrados:               ',len(nits),'\n',
      'Cantidad de fechas de matriculas guardadas: ', len(fechas),'\n',
      'Cantidad de Actividades almacenadas:        ', len(actividades),'\n',
      'Cantidad de Registros No encontrados:       ', len(no_nits), '\n')

Cantidad de NITS encontrados:                42 
 Cantidad de fechas de matriculas guardadas:  42 
 Cantidad de Actividades almacenadas:         42 
 Cantidad de Registros No encontrados:        94 



# Pruebas 

In [8]:
 #Activar WebDriver
    
print('Activando Web Driver...')
url='https://www.rues.org.co/'
options = webdriver.ChromeOptions()
#options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)
driver.get(url)

time.sleep(3) #DELAY

print('Cerrando Ventana Emergente...')
#Cerrar Ventana Emergente Inicialabs
button = driver.find_element("xpath", '/html/body/div[1]/section/div/div/div/div[3]/button')
button.click()

time.sleep(1)

print('Ingresando Nit ', NIT[n])
#Caja Ingreso Nit
input_nit = driver.find_element("xpath", '/html/body/div[1]/main[1]/div/div[2]/div/div/div[2]/form/div/input') # Caja NIT
input_nit.click() # CLICK

time.sleep(1)

input_nit.send_keys(NIT[n]) # INGRESO EL NIT
driver.find_element("xpath",'/html/body/div[1]/main[1]/div/div[2]/div/div/div[2]/form/button').click() # Click en buscar

time.sleep(1)


Activando Web Driver...


C:\Users\lcast\AppData\Local\Temp\ipykernel_19732\3957597237.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../chromedriver', options=options)


Cerrando Ventana Emergente...
Ingresando Nit  808000785


In [9]:

        
print('Seleccionando Razon Social...')
# Seleccionar Primer Nombre o Razon Social Resultante
driver.find_element("xpath",'/html/body/div[1]/main[1]/div/div[6]/div/div/div/div/table/tbody/tr/td[1]').click() 

time.sleep(1)
        


Seleccionando Razon Social...


In [10]:
#Click en Info
# Encontrar y clickear info
try:
    print('Probando Paths 1...')
    info = driver.find_elements( "xpath",'//a[@class="'+"text-success"+'"]') #<a class="text-success" href="javascript:ConsultarDetalleRM('40002161982')"><i class="fa fa-info text-success"></i>nfo</a>
    time.sleep(1)
    try_click_info(info)
except:
    print('no found')
#             try:
#                 print('Probando Paths 2...')
#                 info2 = driver.find_elements( "xpath",'//a[@class="'+"text-muted"+'"]')
#                 time.sleep(1)                         //*[@id="rmTable2"]/tbody/tr[2]/td/ul/li[6]/span[2]/a       
#                 try_click_info(info2)
#             except: 
#                 print('Path info de ', NIT[n], 'no encontrado')
       
#         #Extraer Primera Actividad Economica
#         try:
#             print('Extrayendo Actividad Economica...')
#             actividad = driver.find_element('xpath','/html/body/div[1]/main/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/ul/li[1]')
        
#         except: 
#             time.sleep(1)
#             print('Extrayendo Actividad Economica...')
#             actividad = driver.find_element('xpath','/html/body/div[1]/main/div/div[2]/div[2]/div/div[2]/div[2]/div[2]/ul/li[1]')
        
#         try:
#             print('Extrayendo fecha de Matricula...')
#             fecha_matricula = driver.find_element('xpath','/html/body/div[1]/main/div/div[2]/div[2]/div/div[1]/div/div[2]/div/table/tbody/tr[4]/td[2]')
#             fecha = fecha_matricula.text
#         except:
#             print('Fecha de ',NIT[n], 'No encontrada.')
#         time.sleep(1)
        
#         txt = actividad.text
#         actividades.append(txt)
#         nits.append(NIT[n])
#         fechas.append(fecha)
        
#         print('Guardando actividad, nit y fecha')
#         print('Actividad: ', actividad.text) 
#     except:
#         no_nits.append(NIT[n])  
#         print(NIT[n],'no encontrado')

#     driver.close()

Probando Paths 1...
no found


In [18]:
driver.close()

In [15]:
info = driver.find_elements( "xpath",'//*[@id="rmTable2"]/tbody/tr[2]/td/ul/li[6]/span[2]/a') #<a class="text-success" href="javascript:ConsultarDetalleRM('40002161982')"><i class="fa fa-info text-success"></i>nfo</a>
len(info)

1

In [16]:
info[0].click()

In [ ]:
fecha_matricula = driver.find_element('xpath','/html/body/div[1]/main/div/div[2]/div[2]/div/div[1]/div/div[2]/div/table/tbody/tr[4]/td[2]')
fecha_matricula.text